This version of the snake game allows you to implement and run your evolutionary algorithm

In [1]:
import random
import time
import turtle
import numpy as np
from functools import partial
import operator

from deap import algorithms
from deap import base
from deap import creator
from deap import tools
from deap import gp

In [2]:
XSIZE = YSIZE = 16

Change headless to True when evolving your solution. You can then pickle your final solution, and run it with this set to True to visualize the result, if you wish. Note that, when using Turtle,  to run it again properly you might need to go to restart your kernel after the visualization has finished.

If you are running on the cloud, or on Google Colab, you can chose to remove the turtle and display code altogether and only run headless. You could then pickle your best solution and use this code to see it play.

In [3]:
HEADLESS = True

In [4]:

class DisplayGame:
    def __init__(self, XSIZE, YSIZE):
        # SCREEN
        self.win = turtle.Screen()
        self.win.title("EVCO Snake game")
        self.win.bgcolor("grey")
        self.win.setup(width=(XSIZE*20)+40,height=(YSIZE*20)+40)
        #self.win.screensize((XSIZE*20)+20,(YSIZE*20)+20)
        self.win.tracer(0)

        #Snake Head
        self.head = turtle.Turtle()
        self.head.shape("square")
        self.head.color("black")

        # Snake food
        self.food = turtle.Turtle()
        self.food.shape("circle")
        self.food.color("yellow")
        self.food.penup()
        self.food.shapesize(0.55, 0.55)
        self.segments = []

    def reset(self, snake):
        self.segments = []
        self.head.penup()
        self.food.goto(-500, -500)
        self.head.goto(-500, -500)
        for i in range(len(snake)-1):
            self.add_snake_segment()
        self.update_segment_positions(snake)

    def update_food(self,new_food):
        self.food.goto(((new_food[1]-9)*20)+20, (((9-new_food[0])*20)-10)-20)

    def update_segment_positions(self, snake):
        self.head.goto(((snake[0][1]-9)*20)+20, (((9-snake[0][0])*20)-10)-20)
        for i in range(len(self.segments)):
            self.segments[i].goto(((snake[i+1][1]-9)*20)+20, (((9-snake[i+1][0])*20)-10)-20)

    def add_snake_segment(self):
        self.new_segment = turtle.Turtle()
        self.new_segment.speed(0)
        self.new_segment.shape("square")
        self.new_segment.color(random.choice(['blue']))
        self.new_segment.penup()
        self.segments.append(self.new_segment)


In [5]:

class snake:
    def __init__(self, _XSIZE, _YSIZE):
        self.XSIZE = _XSIZE
        self.YSIZE = _YSIZE
        self.reset()

    def reset(self):
        self.snake = [[8,10], [8,9], [8,8], [8,7], [8,6], [8,5], [8,4], [8,3], [8,2], [8,1],[8,0] ]# Initial snake co-ordinates [ypos,xpos]
        self.food = self.place_food()
        self.ahead = []
        self.newfood=[]
        self.Lhead=[]
        self.Rhead=[]
        self.snake_direction = "right"
    def place_food(self):
        self.food = [random.randint(1, (YSIZE-2)), random.randint(1, (XSIZE-2))]
        while (self.food in self.snake):
            self.food = [random.randint(1, (YSIZE-2)), random.randint(1, (XSIZE-2))]
        return(self.food)


    def update_snake_position(self):
        self.snake.insert(0, [self.snake[0][0] + (self.snake_direction == "down" and 1) + (self.snake_direction == "up" and -1), self.snake[0][1] + (self.snake_direction == "left" and -1) + (self.snake_direction == "right" and 1)])

    def food_eaten(self):
        if self.snake[0] == self.food:                                            # When snake eats the food
            return True
        else:
            last = self.snake.pop()  # [1] If it does not eat the food, it moves forward and so last tail item is removed
            return False

    def snake_turns_into_self(self):
        if self.snake[0] in self.snake[1:]:
            return True
        else:
            return False

    def snake_hit_wall(self):
        if self.snake[0][0] == 0 or self.snake[0][0] == (YSIZE-1) or self.snake[0][1] == 0 or self.snake[0][1] == (XSIZE-1):
            return True
        else:
            return False


## Movement Terminals ##

    def turn_left(self):
        self.snake_direction = "left"

    def turn_right(self):
        self.snake_direction = "right"

    def go_forward(self):
        pass

    def turn_up(self):
        self.snake_direction = "up"

    def turn_down(self):
        self.snake_direction = "down"

## Get Left Right Coordinates and Ahead ##
    def getAheadLocation(self):
        self.ahead = [ self.snake[0][0] + (self.snake_direction == "down" and 1) + (self.snake_direction == "up" and -1), self.snake[0][1] + (self.snake_direction == "left" and -1) + (self.snake_direction == "right" and 1)]

    def getLeftLocation(self):
        self.Lhead = [ self.snake[0][0] + (self.snake_direction == "right" and -1) + (self.snake_direction == "left" and 1), self.snake[0][1] + (self.snake_direction == "up" and -1) + (self.snake_direction == "down" and 1)]

    def getRightLocation(self):
        self.Rhead = [ self.snake[0][0] + (self.snake_direction == "right" and 1) + (self.snake_direction == "left" and -1), self.snake[0][1] + (self.snake_direction == "up" and 1) + (self.snake_direction == "down" and -1)]

## Sense Wall ##

    def sense_wall_ahead(self):
        self.getAheadLocation()

        return( self.ahead[0] == 0 or self.ahead[0] == (YSIZE-1) or self.ahead[1] == 0 or self.ahead[1] == (XSIZE-1) )

    def sense_wall_Left(self):
        self.getLeftLocation()
        return( self.Lhead[0] == 0 or self.Lhead[0] == (YSIZE-1) or self.Lhead[1] == 0 or self.Lhead[1] == (XSIZE-1) )

    def sense_wall_Right(self):
        self.getRightLocation()
        return( self.Rhead[0] == 0 or self.Rhead[0] == (YSIZE-1) or self.Rhead[1] == 0 or self.Rhead[1] == (XSIZE-1) )




## Sense Food ##

    def sense_food_ahead(self):
        self.getAheadLocation()
        return self.food == self.ahead

    def sense_food_Left(self):
        self.getLeftLocation()
        return self.food == self.Lhead

    def sense_food_Right(self):
        self.getRightLocation()
        return self.food == self.Rhead


## Sense moving ##

    def sense_moving_up(self):
        return (self.snake_direction == "up")

    def sense_moving_down(self):
        return self.snake_direction == 'down'

    def sense_moving_right(self):
        return self.snake_direction == 'right'

    def sense_moving_left(self):
        return self.snake_direction == 'left'


## Sense object ##

    def sense_object_ahead(self):
        return self.sense_wall_ahead() or self.sense_tail_ahead()

    def sense_object_Left(self):
        return self.sense_wall_Left() or self.sense_tail_Left()

    def sense_object_Right(self):
        return self.sense_wall_Right() or self.sense_tail_Right()

## Sense Tail ##
    def sense_tail_ahead(self):
        self.getAheadLocation()
        return self.ahead in self.snake

    def sense_tail_Left(self):
        self.getLeftLocation()
        return self.Lhead in self.snake

    def sense_tail_Right(self):
        self.getRightLocation()
        return self.Rhead in self.snake


## 'If' Moving ##
    def if_moving_up(self, out1, out2):
        return partial(if_then_else, self.sense_moving_up, out1, out2)

    def if_moving_down(self, out1, out2):
        return partial(if_then_else, self.sense_moving_down, out1, out2)

    def if_moving_right(self, out1, out2):
        return partial(if_then_else, self.sense_moving_right, out1, out2)

    def if_moving_left(self, out1, out2):
        return partial(if_then_else, self.sense_moving_left, out1, out2)

## 'If' Food ##
    def if_food_ahead(self, out1, out2):
        return partial(if_then_else, self.sense_food_ahead, out1, out2)

    def if_food_Left(self, out1, out2):
        return partial(if_then_else, self.sense_food_Left, out1, out2)

    def if_food_Right(self, out1, out2):
        return partial(if_then_else, self.sense_food_Right, out1, out2)

### 'If' Tail ###
    def if_tail_ahead(self, out1, out2):
        return partial(if_then_else, self.sense_tail_ahead, out1, out2)

    def if_tail_Left(self, out1, out2):
        return partial(if_then_else, self.sense_tail_Left, out1, out2)

    def if_tail_Right(self, out1, out2):
        return partial(if_then_else, self.sense_tail_Right, out1, out2)

### 'If' Wall ###
    def if_wall_ahead(self, out1, out2):
        return partial(if_then_else, self.sense_wall_ahead, out1, out2)


    def if_wall_Left(self, out1, out2):
        return partial(if_then_else, self.sense_wall_Left, out1, out2)

    def if_wall_Right(self, out1, out2):
        return partial(if_then_else, self.sense_wall_Right, out1, out2)

    def if_wall_Bottom(self, out1, out2):
        return partial(if_then_else, self.sense_wall_Bottom, out1, out2)

##  'If' Object ##
    def if_object_ahead(self, out1, out2):
        return partial(if_then_else, self.sense_object_ahead, out1, out2)

    def if_object_Left(self, out1, out2):
        return partial(if_then_else, self.sense_object_Left, out1, out2)

    def if_object_Right(self, out1, out2):
        return partial(if_then_else, self.sense_object_Right, out1, out2)
##  Food senses ##

    def add_food(self,first,secound):
        a=first[0]+secound[0]
        b=first[1]+secound[1]
        if (first[0]+secound[0] >= XSIZE-2):
            a = YSIZE - 2
        if (first[1]+secound[1] >= YSIZE-2):
            b = YSIZE - 2
        return [a,b]

    def subtract_food(self,first,secound):
        a=first[0]-secound[0]
        b=first[1]-secound[1]
        if (first[0]-secound[0] <= 1):
            a = 1
        if (first[1]-secound[1] <= 1):
            b = 1
        return [a,b]



In [6]:
snake_game = snake(XSIZE,YSIZE)

In [7]:

def run_game(individualS,individualF,display,snake_game, headless):

    score = 0
    steps = 0
    total = 0
    snake_game.reset()
    if not headless:
        display.reset(snake_game.snake)
        display.win.update()
    func = toolbox2.compile(expr=individualF)
    snake_game.food=func(snake_game.snake[0])
    while (snake_game.food in snake_game.snake):## uses random input for function if places food on snake
        snake_game.food=func([random.randint(1, (YSIZE-2)), random.randint(1, (XSIZE-2))])
        snake_game.place_food()
    game_over = False
    snake_direction = "right"
    Snake_GP = gp.compile(individualS, pset)

    flag = True

    while not game_over:
        Snake_GP()
        if steps==200:
            game_over=True
        # Here is an example sensing function
        #if snake_game.sense_tail_ahead():
           # print("Tail ahead!!!!")
        # ****YOUR AI ABOVE HERE******************
        snake_game.update_snake_position()

        # Check if food is eaten
        if snake_game.food_eaten():
            func = toolbox2.compile(expr=individualF)
            snake_game.food=func(snake_game.snake[0])
            while (snake_game.food in snake_game.snake): ## uses random input for function if places food on snake
                snake_game.food=func([random.randint(1, (YSIZE-2)), random.randint(1, (XSIZE-2))]) 
                snake_game.place_food()
            steps=0
            score += 1
            if not headless: display.add_snake_segment()

        # Game over if the snake runs over itself
        if snake_game.snake_turns_into_self():
            game_over = True


        # Game over if the snake goes through a wall
        if snake_game.snake_hit_wall():
            game_over = True


        if not headless:
            display.update_food(snake_game.food)
            display.update_segment_positions(snake_game.snake)
            display.win.update()
             # Change this to modify the speed the game runs at when displayed.

            time.sleep(0.005)

        steps += 1
        total += 1

    #print("\nFINAL score - " + str(score))
    turtle.clearscreen()
    return (score,total)

In [8]:

def progn(*args):
    for arg in args:
        arg()

def prog2(out1, out2):
    return partial(progn,out1,out2)

def prog3(out1, out2, out3):
    return partial(progn,out1,out2,out3)

def if_then_else(condition, out1, out2):
    out1() if condition() else out2()


In [9]:

##  Snake Genetic Programming ##

pset = gp.PrimitiveSet("MAIN", 0)

pset.addPrimitive(prog2, 2)
pset.addPrimitive(prog3, 3)

pset.addPrimitive(snake_game.if_moving_up, 2)
pset.addPrimitive(snake_game.if_moving_down, 2)
pset.addPrimitive(snake_game.if_moving_left, 2)
pset.addPrimitive(snake_game.if_moving_right, 2)

pset.addPrimitive(snake_game.if_food_ahead,2)
pset.addPrimitive(snake_game.if_food_Left,2)
pset.addPrimitive(snake_game.if_food_Right,2)

pset.addPrimitive(snake_game.if_object_ahead,2)
pset.addPrimitive(snake_game.if_object_Left,2)
pset.addPrimitive(snake_game.if_object_Right,2)

pset.addTerminal(snake_game.turn_left)
pset.addTerminal(snake_game.go_forward)
pset.addTerminal(snake_game.turn_right)
pset.addTerminal(snake_game.turn_down)
pset.addTerminal(snake_game.turn_up)


In [10]:
## Food Placing Genetic Programming ##

pset2 = gp.PrimitiveSet("MAIN",1)


for i in range(1,15):
    for j in range(1,15):
        pset2.addTerminal([i,j])

pset2.addPrimitive(snake_game.add_food, 2)
pset2.addPrimitive(snake_game.subtract_food, 2)

In [11]:
## GP for snake ##

creator.create("FitnessMax", base.Fitness, weights=(2.0,1))
creator.create("Individual", gp.PrimitiveTree, fitness=creator.FitnessMax,pset=pset)
#creator.create("IndividualC", gp.PrimitiveTree, fitness=creator.FitnessMax,pset=pset2)
toolbox = base.Toolbox()

toolbox.register("expr_init", gp.genHalfAndHalf, pset=pset, min_=1, max_=4)
toolbox.register("individual", tools.initIterate, creator.Individual, toolbox.expr_init)
#toolbox.register("select", tools.selDoubleTournament, fitness_size = 10, parsimony_size = 1.4, fitness_first = True, fit_attr='fitness' )
toolbox.register("select", tools.selTournament,tournsize=10,fit_attr='fitness')
toolbox.register("mate", gp.cxOnePoint)
toolbox.register("expr_mut", gp.genHalfAndHalf, min_=0, max_=3)
toolbox.register("mutate", gp.mutUniform, expr=toolbox.expr_mut, pset=pset)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)


In [12]:

## GP for Food ##

creator.create("FitnessMaxF", base.Fitness, weights=(1.0,2))
creator.create("IndividualFood", gp.PrimitiveTree, fitness=creator.FitnessMaxF,pset=pset2)

toolbox2 = base.Toolbox()
toolbox2.register("expr_init", gp.genHalfAndHalf, pset=pset2, min_=1, max_=4)
toolbox2.register("individual", tools.initIterate, creator.IndividualFood, toolbox2.expr_init)
toolbox2.register("select", tools.selTournament,tournsize=10,fit_attr='fitness')
toolbox2.register("mate", gp.cxOnePoint)
toolbox2.register("expr_mut", gp.genHalfAndHalf, min_=0, max_=3)
toolbox2.register("mutate", gp.mutUniform, expr=toolbox2.expr_mut, pset=pset2)
toolbox2.register("population", tools.initRepeat, list, toolbox2.individual)
toolbox2.register("compile", gp.compile, pset=pset2)

In [13]:
##Static Limit to prevent bloating
toolbox.decorate("mate", gp.staticLimit(key=operator.attrgetter("height"), max_value=15))
toolbox.decorate("mutate", gp.staticLimit(key=operator.attrgetter("height"), max_value=15))

toolbox2.decorate("mate", gp.staticLimit(key=operator.attrgetter("height"), max_value=15))
toolbox2.decorate("mutate", gp.staticLimit(key=operator.attrgetter("height"), max_value=15))

In [14]:
NUM_OPPONENTS = 2

In [15]:

def evaluate(pop_S,pop_F):
    Fitnesses_S = [(0.0,0,0)] * len(pop_S)
    Fitnesses_F = [(0.0,0.0)] * len(pop_F)

    PlayOrderS = list(range(len(pop_S)))
    PlayOrderF = list(range(len(pop_F)))
    for i in list(range(NUM_OPPONENTS)):
        # Randomize the order of pop L to play against corresponding individuals in pop R
        random.shuffle(PlayOrderS)

        for i_L, i_R in zip(PlayOrderS, PlayOrderF):
            IndividualS = pop_S[i_L]
            IndividualF = pop_F[i_R]
            display = DisplayGame(XSIZE,YSIZE)
            (score,steps) = run_game(IndividualS,IndividualF,display,snake_game, headless=HEADLESS)
            Fitnesses_S[i_L] = (score,steps)
            Fitnesses_F[i_R] = (-score,-steps)

    # Careful here, we are returning the lists of fitnesses of both pops
    return Fitnesses_S, Fitnesses_F



toolbox.register("evaluate", evaluate)


In [16]:
stats = tools.Statistics(key=lambda ind: ind.fitness.values)
stats.register("avg", np.mean)
stats.register("std", np.std)
stats.register("min", np.min)
stats.register("max", np.max)

In [17]:

NGEN, CXPB, MUTPB = 40, 0.3, 0.5



LOGLIST_SNAKE =[]

LOGLIST_FOOD=[]

NumberOfModels = 1 ## Change this to increase number of runs of the algorithm (more runs for better average)
for i in range(NumberOfModels): 
    print("Model version ", i+1)
    popS = toolbox.population(n=100)
    popF = toolbox2.population(n=100)

    fitnessesL, fitnessesR = toolbox.evaluate(popS, popF)
    for ind, (fit,fit2) in zip(popS, fitnessesL):
        ind.fitness.values = (fit,fit2)

    for ind, (fit,fit2) in zip(popF, fitnessesR):
        ind.fitness.values = (fit,fit2)

    logbookS = tools.Logbook()
    recordS = stats.compile(popS)
    logbookS.record(gen=0, **recordS)

    logbookF = tools.Logbook()
    recordF = stats.compile(popF)
    logbookF.record(gen=0, **recordF)
    for g in range(NGEN):
        print("-- Generation %i --" % g)
        if g == 41: ## Change this to turn headless off
            HEADLESS = False

    ##  Snake evolution ##
        selectedS = toolbox.select(popS, len(popS))
        offspringS = list(map(toolbox.clone, selectedS))

        for child1, child2 in zip(offspringS[::2], offspringS[1::2]):
            if random.random() < CXPB:
                toolbox.mate(child1, child2)
                del child1.fitness.values
                del child2.fitness.values

        for mutantS in offspringS:
            if random.random() < MUTPB:
                toolbox.mutate(mutantS)
        popS[:] = offspringS

    ## Food evolution ##
        selectedF = toolbox.select(popF, len(popF))
        offspringF = list(map(toolbox.clone, selectedF))

        for child1, child2 in zip(offspringF[::2], offspringF[1::2]):
            if random.random() < CXPB:
                toolbox2.mate(child1, child2)
                del child1.fitness.values
                del child2.fitness.values

        for mutantF in offspringF:
            if random.random() < MUTPB:
                toolbox2.mutate(mutantF)
        popF[:] = offspringF


        fitnessesL, fitnessesR = toolbox.evaluate(popS, popF)

        for ind, (fit,fit2) in zip(popS, fitnessesL):
            ind.fitness.values = (fit,fit2)

        for ind, (fit,fit2) in zip(popF, fitnessesR):
            ind.fitness.values = (fit,fit2)

        recordS = stats.compile(popS)
        logbookS.record(gen=g, **recordS)

        recordF = stats.compile(popF)
        logbookF.record(gen=g, **recordF)
        print(logbookS.stream)
        print(logbookF.stream)
    LOGLIST_SNAKE.append(logbookS)
    LOGLIST_FOOD.append(logbookF)

Model version  1
-- Generation 0 --
avg 	gen	max	min	std    
2.91	0  	20 	0  	3.63344
4.71	0  	20 	0  	5.62547
avg  	gen	max	min	std    
-2.91	0  	0  	-20	3.63344
-4.71	0  	0  	-20	5.62547
-- Generation 1 --
6.73	1  	20 	0  	7.90867
-6.73	1  	0  	-20	7.90867
-- Generation 2 --
6.75	2  	20 	0  	7.90364
-6.75	2  	0  	-20	7.90364
-- Generation 3 --
8.145	3  	20 	0  	9.10077
-8.145	3  	0  	-20	9.10077
-- Generation 4 --
8.775	4  	20 	0  	9.57833
-8.775	4  	0  	-20	9.57833
-- Generation 5 --
8.725	5  	20 	0  	9.58746
-8.725	5  	0  	-20	9.58746
-- Generation 6 --
8.745	6  	20 	0  	9.47787
-8.745	6  	0  	-20	9.47787
-- Generation 7 --
8.68 	7  	20 	0  	9.51565
-8.68 	7  	0  	-20	9.51565
-- Generation 8 --
8.785	8  	20 	0  	9.58795
-8.785	8  	0  	-20	9.58795
-- Generation 9 --
8.63 	9  	20 	0  	9.45955
-8.63 	9  	0  	-20	9.45955
-- Generation 10 --
8.865	10 	20 	0  	9.46714
-8.865	10 	0  	-20	9.46714
-- Generation 11 --
8.18 	11 	20 	0  	9.30686
-8.18 	11 	0  	-20	9.30686
-- Generation 12 --
9

TclError: invalid command name ".!canvas"

In [ ]:
import math
import matplotlib.pyplot as plt

gen = logbookF.select("gen")
genM = logbookS.select("max")


plt.rc('axes', labelsize=14)
plt.rc('xtick', labelsize=14)
plt.rc('ytick', labelsize=14)
plt.rc('legend', fontsize=14)



## Find average and standard deviation of snake models
meantotal_snake=[]
for i in range(NumberOfModels):
    meantotal_snake.append(LOGLIST_SNAKE[i].select("avg"))

standardtotal_snake=[]
for i in range(NumberOfModels):
    standardtotal_snake.append(LOGLIST_SNAKE[i].select("std"))
    
mean_average_snake = [sum(sub_list) / len(sub_list) for sub_list in zip(*meantotal_snake)]


def square(a):
    new=[]
    for i in range(len(a)):
        new.append(a[i] ** 2)
    return new

standard_average_snake = [math.sqrt(sum(square(sub_list))) / len(sub_list) for sub_list in zip(*standardtotal_snake)]


## Find average and standard deviation of Food models
meantotal_food=[]
for i in range(NumberOfModels):
    meantotal_food.append(LOGLIST_FOOD[i].select("avg"))

standardtotal_food=[]
for i in range(NumberOfModels):
    standardtotal_food.append(LOGLIST_FOOD[i].select("std"))
    
mean_average_food = [sum(sub_list) / len(sub_list) for sub_list in zip(*meantotal_food)]

import math

def square(a):
    new=[]
    for i in range(len(a)):
        new.append(a[i] ** 2)
    return new

standard_average_food = [math.sqrt(sum(square(sub_list))) / len(sub_list) for sub_list in zip(*standardtotal_food)]
#Plot on line Graph with error bars

fig, ax1 = plt.subplots()
line1 = plt.errorbar(gen, mean_average_snake, standard_average_snake,label='Algorithm Snake',color = 'orange')
line2 = plt.errorbar(gen, mean_average_food, standard_average_food,label='Algorithm Food',color = 'blue')
ax1.set_xlabel("Generation")
ax1.set_ylabel("Fitness")

plt.legend(loc='best', fancybox=True, framealpha=0.5)
plt.show()


In [ ]:
##Saved these results from previous runs 
data_Crossover = [2.6275000000000004, 4.103, 5.544, 6.355000000000001, 6.784999999999999, 7.1125, 7.433000000000002, 8.2905, 8.568000000000001, 9.3605, 9.802499999999998, 10.393500000000001, 11.047, 12.270500000000002, 13.823999999999998, 14.577000000000002, 16.0515, 18.7115, 19.166999999999998, 18.8985, 19.45]
data_Mutation = [2.8019999999999996, 4.073, 5.276000000000001, 6.117, 6.497000000000002, 7.627, 8.225999999999999, 8.587, 9.194999999999999, 10.195, 11.165000000000003, 11.450000000000001, 11.523, 11.793000000000003, 12.145999999999999, 12.863, 12.754000000000001, 12.979, 14.060000000000002, 13.779, 14.219999999999999, 14.417000000000002, 14.161000000000001, 14.592999999999998, 14.65, 14.797, 14.978, 14.938999999999998, 14.694999999999999, 15.641000000000002, 16.339000000000002, 17.595000000000002, 17.666, 18.066, 18.726999999999997, 19.024, 19.198, 19.583, 20.291, 20.805, 19.506999999999998] 
data_Population = [2.8165, 4.337, 5.606, 6.118500000000001, 6.846500000000001, 7.477500000000001, 8.062, 8.6125, 9.7815, 11.060999999999998, 13.994500000000002, 17.5175, 20.946, 24.395000000000003, 26.165999999999997, 29.325, 28.318499999999993, 28.677999999999997, 29.4995, 30.903000000000002, 31.177000000000003, 32.43300000000001, 31.77, 31.192500000000003, 31.096500000000002, 29.531, 30.39549999999999, 29.761499999999995, 29.838, 29.568, 28.035499999999995, 27.3615, 28.371500000000005, 30.539000000000005, 31.311999999999994, 30.836000000000002, 30.68900000000001, 32.17399999999999, 30.939, 31.496999999999996, 29.832]

data = [data_Crossover, data_Mutation, data_C]

fig = plt.figure(figsize =(10, 7)) 
ax = fig.add_axes([0, 0, 1, 1])
ax.set_xticklabels(['High Crossover', 'High Mutation','High Population'])

ax.set_xlabel('Condition')
ax.set_ylabel('Mean Fitness')

bp = ax.boxplot(data)

In [ ]:
from scipy.stats import mannwhitneyu
stat, p = mannwhitneyu(data_A, data_B)
print('Statistics=%.3f, p=%.3f' % (stat, p))